In [120]:
import pandas as pd
import numpy as np

In [122]:
np.__version__

'1.21.5'

# Predicting heart disease using machine learning
This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting whether or not someone has heart disease based on their medical attributes.

We're going to take the following approach:

Problem definition
Data
Evaluation
Features
Modelling
Experimentation
## 1. Problem Definition
In a statement,

Given clinical parameters about a patient, can we predict whether or not they have heart disease?

## 2. Data
The original data came from the Cleavland data from the UCI Machine Learning Repository. https://archive.ics.uci.edu/ml/datasets/heart+Disease

There is also a version of it available on Kaggle. https://www.kaggle.com/datasets/sumaiyatasmeem/heart-disease-classification-dataset

## 3. Evaluation
if we can reach 95% accuracy at predicting whether or not a patient has heart disease during the proof of concept, we'll pursue the project.

## 4. Features
This is where you'll get different information about each of the features in your data. You can do this via doing your own research (such as looking at the links above) or by talking to a subject matter expert (someone who knows about the dataset).

#### Create data dictionary

1.age - age in years         
2.sex - (1 = male; 0 = female)       
3.cp - chest pain type 
          0: Typical angina: chest pain related decrease blood supply to the heart     
          1: Atypical angina: chest pain not related to heart                 
          2: Non-anginal pain: typically esophageal spasms (non heart related)            
          3: Asymptomatic: chest pain not showing signs of disease                  
4.trestbps - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause for concern       
5.chol - serum cholestoral in mg/dl       
          serum = LDL + HDL + .2 * triglycerides           
          above 200 is cause for concern           
6.fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)          
           '>126' mg/dL signals diabetes                       
7.restecg - resting electrocardiographic results             
            0: Nothing to note                     
            1: ST-T Wave abnormality                      
                    can range from mild symptoms to severe problems                       
                    signals non-normal heart beat                        
            2: Possible or definite left ventricular hypertrophy           
                     Enlarged heart's main pumping chamber                   
8.thalach - maximum heart rate achieved                 
9.exang - exercise induced angina (1 = yes; 0 = no)                         
10.oldpeak - ST depression induced by exercise relative to rest looks at stress of heart during excercise unhealthy heart will stress more                     
11.slope - the slope of the peak exercise ST segment                  
                       0: Upsloping: better heart rate with excercise (uncommon)                
                       1: Flatsloping: minimal change (typical healthy heart)                
                       2: Downslopins: signs of unhealthy heart           
12.ca - number of major vessels (0-3) colored by flourosopy           
                           colored vessel means the doctor can see the blood passing through                 
                           the more blood movement the better (no clots)                     
13.thal - thalium stress result            
          1,3: normal                           
          6: fixed defect: used to be defect but ok now               
          7: reversable defect: no proper blood movement when excercising                
14.target - have disease or not (1=yes, 0=no) (= the predicted attribute)                                            

# Preparing the tools
We're going to use pandas, Matplotlib and NumPy for data analysis and manipulation.

In [ ]:
# Import all the tools that we need

# Regular EDA(Exploratory data analysis) and plotting library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# We want our plots to appear inline
%matplotlib inline

# Models from Scikit-Learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import RocCurveDisplay


# Load Data

In [ ]:
df = pd.read_csv('heart-disease.csv')
df

In [ ]:
df.shape  # rows and columns

# Data Exploration (exploratory data analysis or EDA)
The goal here is to find out more about the data and become a subject matter export on the dataset you're working with.

What question(s) are you trying to solve?          
What kind of data do we have and how do we treat different types?             
What's missing from the data and how do you deal with it?                  
Where are the outliers and why should you care about them?              
How can you add, change or remove features to get more out of your data?               

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Let's find out how many of each class there
df['target'].value_counts()

In [ ]:
df['target'].value_counts().plot(kind = 'bar', color = ['salmon','lightblue'])

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

## Heart Disease Frequency according to Sex

In [ ]:
df.sex.value_counts()

In [ ]:
# Compare the target column with the sex column
pd.crosstab(df.target, df.sex)

In [ ]:
pd.crosstab(df.target, df.sex).plot(kind = 'bar', figsize = (10, 6), color = ['salmon', 'lightblue'])
plt.title('Heart Disease Frequency according to Sex')
plt.xlabel("0 = No Diesease, 1 = Disease")
plt.ylabel("Amount")
plt.legend(['Female', 'Male']);
plt.xticks(rotation = 0);

## Age vs. Max Heart Rate for Heart Disease

In [ ]:
# Create another figure
plt.figure(figsize= (10,6))

# Scatter with postivie examples
plt.scatter(df.age[df.target == 1],
           df.thalach[df.target == 1],
           c = 'salmon')

# Scatter with negative examples
plt.scatter(df.age[df.target == 0],
           df.thalach[df.target == 0],
           c = 'lightblue')

plt.title('Age vs. Max Heart Rate for Heart Disease')
plt.xlabel("Age")
plt.ylabel("Max heart rate")
plt.legend(['Disease', 'No Disease']);

In [ ]:
# Check the distribution of the age column with a histogram
df.age.plot.hist()

## Heart Disease Frequency per Chest Pain Type

cp - chest pain type                                                    
0: Typical angina: chest pain related decrease blood supply to the heart                              
1: Atypical angina: chest pain not related to heart                               
2: Non-anginal pain: typically esophageal spasms (non heart related)                      
3: Asymptomatic: chest pain not showing signs of disease                          

In [ ]:
pd.crosstab(df.cp, df.target)

In [ ]:
# Make the cross tab more visible
pd.crosstab(df.cp, df.target).plot(kind = 'bar', figsize = (10,6), color =['salmon','lightblue'])

# Add some communication
plt.title('Heart Disease Frequency per Chest Pain Type')
plt.xlabel("Chest Pain")
plt.ylabel("Amount")
plt.legend(['No Disease', 'Disease']);
plt.xticks(rotation = 0);

In [ ]:
df.head()

In [ ]:
# Make a Correlation matrix
df.corr()

In [ ]:
# Lets make our correlation matrix a little prettier
corr_matrix = df.corr()
fig, ax = plt.subplots(figsize = (10, 6))
ax = sns.heatmap(corr_matrix,
                annot = True,
                linewidths = 0.5,
                fmt = '.2f',
                cmap = 'YlGnBu');
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

## 5. Modelling

In [ ]:
df.head()

In [ ]:
# Split data into X and y
X = df.drop('target', axis = 1)

y = df['target']

In [ ]:
X

In [ ]:
y

In [ ]:
# Split data into test and train set
np.random.seed(42)

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train

In [ ]:
y_train, len(y_train)

Now we've got our data split into training and test sets, it's time to build a machine learning model.        
                                                       
We'll train it (find the patterns) on the training set.                            
                                                               
And we'll test it (use the patterns) on the test set.                                         
                                                      
We're going to try 3 different machine learning models:                                     
                         
Logistic Regression                
K-Nearest Neighbours Classifier                       
Random Forest Classifier                      

In [ ]:
# Put modes in a dictionary
models = {'Logistic Regression': LogisticRegression(),
         'KNN': KNeighborsClassifier(),
         'Random Forest Classifier': RandomForestClassifier()}
# Create a function to fit a model Score
def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given machine learning models.
    models : a dict of differetn Scikit-Learn machine learning models
    X_train : training data (no labels)
    X_test : testing data (no labels)
    y_train : training labels
    y_test : test labels
    """
    # Set random seed
    np.random.seed(42)
    # Make a dictionary to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train,y_train)
        # Evaluate the model and append the scores to model_scores
        model_scores[name] = model.score(X_test, y_test)
    return model_scores
        
        

In [ ]:
model_scores = fit_and_score(models = models,
                            X_train = X_train,
                            X_test = X_test,
                            y_train = y_train,
                            y_test=y_test)
model_scores

## Model Comparison 

In [ ]:
model_compare = pd.DataFrame(model_scores, index=['accuracy'])
model_compare.T.plot.bar()    #T for transpose

Now we've got a baseline model... and we know a model's first predictions aren't always what we should based our next steps off. What should we do?           

Let's look at the following:           

Hypyterparameter tuning               
Feature importance                      
Confusion matrix                          
Cross-validation                   
Precision                     
Recall                  
F1 score                     
Classification report                   
ROC curve                           
Area under the curve (AUC)                        


# Hyperparameter tuning (by hand)

In [ ]:
# Let's tune KNN
train_scores = []
test_scores = []

# Create a list of diff values for n_neighbors
neighbors = range(1,21)

# Set up KNN instance
knn = KNeighborsClassifier()

# Loop through different n_neighbors
for i in neighbors:
    knn.set_params(n_neighbors = i)
    
    #Fit the algorithm
    knn.fit(X_train, y_train)
    
    #Update the training scores list
    train_scores.append(knn.score(X_train, y_train))
    
    # Update the test scores list
    test_scores.append(knn.score(X_test, y_test))

In [ ]:
X_train

In [ ]:
train_scores

In [ ]:
test_scores

In [ ]:
plt.plot(neighbors, train_scores, label="Train score")
plt.plot(neighbors, test_scores, label="Test score")
plt.xticks(np.arange(1, 21, 1))
plt.xlabel("Number of neighbors")
plt.ylabel("Model score")
plt.legend()

print(f"Maximum KNN score on the test data: {max(test_scores)*100:.2f}%")

# Hyperparameter tuning with RandomizedSearchCV
We're going to tune:             
                             
LogisticRegression()             
RandomForestClassifier()                          
... using RandomizedSearchCV             

In [ ]:
# Create a hyperparameter grid for LogisticRegression
log_reg_grid = {"C": np.logspace(-4, 4, 20),
                "solver": ["liblinear"]}

# Create a hyperparameter grid for RandomForestClassifier
rf_grid = {"n_estimators": np.arange(10, 1000, 50),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2)}

Now we've got hyperparameter grids setup for each of our models, let's tune them using RandomizedSearchCV

In [ ]:
# Tune LogisticRegression

np.random.seed(42)

# Setup random hyperparameter search for LogisticRegression
rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid,
                                cv=5,
                                n_iter=20,
                                verbose=True)

# Fit random hyperparameter search model for LogisticRegression
rs_log_reg.fit(X_train, y_train)

In [ ]:
rs_log_reg.best_params_

Now we've tuned LogisticRegression(), let's do the same for RandomForestClassifier()

In [ ]:
rs_log_reg.score(X_test, y_test)

In [ ]:
# Setup random seed
np.random.seed(42)

# Setup random hyperparameter search for RandomForestClassifier
rs_rf = RandomizedSearchCV(RandomForestClassifier(), 
                           param_distributions=rf_grid,
                           cv=5,
                           n_iter=20,
                           verbose=True)

# Fit random hyperparameter search model for RandomForestClassifier()
rs_rf.fit(X_train, y_train)

In [ ]:
# Find the best hyperparameters
rs_rf.best_params_

In [ ]:
# Evaluate the randomized search RandomForestClassifier model
rs_rf.score(X_test, y_test)